# Projet d'informatique
## Groupe:  Simplicial complexes for topological data analysis
### GNABEYEU Emmanuel
### CHAMOUN Yorgo 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import csv
import scipy
import math as m
import itertools
import time
import random

## Complexe de Čech

### Question 1.

On commence par écrire une fonction qui calcule la distance euclidienne entre deux points de l'espace en dimension quelconque, et qui sera bien utile par la suite.


In [2]:
def dist(x,y):
    return m.sqrt(np.sum([(x[i]-y[i])**2 for i in range(len(x))]))

Comme suggéré par l'énoncé, nous écrivons une fonction qui calcule le diamètre et le centre de la plus petite boule ayant ces points sur sa frontière, ce qui revient à résoudre un système d'équations linéaire développé dans le rapport. Lorsque le système ne peut pas être résolu (lorsque la matrice n'est pas inversible), nous renvoyons un tuple vide et un rayon de -1.

In [3]:
def circum(p):
    d=len(p[0])
    k=len(p)
    a=np.array([[p[i][j]-p[0][j]  for j in range(d)]+[0 for l in range(k-1)] for i in range(1,k)]+[[0 for i in range(j)]+[1]+[0 for i in range(d-j-1)]+[-p[i][j]+p[k-1][j] for i in range(k-1)] for j in range(d)])
    b=np.array([sum([(p[i][j]*p[i][j]-p[0][j]*p[0][j])/2 for j in range(d)]) for i in range(1,k)]+[p[k-1][i] for i in range(d)])
    c=()
    try:
        x = np.linalg.solve(a, b)
    except np.linalg.LinAlgError:
         return c,-1
    for i in range(d):
        c+=(sum([x[d+j]*(p[j][i]-p[k-1][i]) for j in range(k-1)])+p[k-1][i],)
    return c,dist(c,p[0])

Enfin, nous écrivons la fonction qui calcule le centre et le rayon de la MEB, en utilisant un algorithme de type Matousek, Sharir et Welzl.

In [4]:
def MEB_aux(S,C,d):
    if len(S)==0 or len(C)==d+1:    #ce qui revient à dire que S=C,  mais comme nous enlevons des éléments
        return C                    #de la base dans S pour choisir x, nous l'écrivons comme ça 
    x=random.choice(S)
    while x in C:
        S.remove(x)
        x=random.choice(S)
    S.remove(x)
    B=MEB_aux(S, C,d)
    if B!=[] and dist(x,circum(B)[0])<=circum(B)[1]:  #la condition f(B)=f(B∪{x}) s'écrit 
        return B                                      #"x appartient au  cercle ayant les points de B sur sa frontière"
    C.append(x)
    return MEB_aux(S, C,d)
        

def MEB(points):
    d=len(points[0])                        #c'est juste pour ne pas la recalculer à chaque fois                  
    return circum(MEB_aux(points, [], d))


Nous effectuons les tests demandés.

In [5]:
print([(0,0,0)]," -> ",MEB([(0,0,0)]))
print([(-1,0,0),(1,0,0)]," -> ",MEB([(-1,0,0),(1,0,0)]))
print([(-10,0,0), (10,0,0), (0,1,0)]," -> ",MEB([(-10,0,0), (10,0,0), (0,1,0)]))
print([(-5,0,0), (3,-4,0), (3,4,0)]," -> ",MEB([(-5,0,0), (3,-4,0), (3,4,0)]))
print([(5,0,1), (-1,-3,4), (-1,-4,-3), (-1,4,-3)]," -> ",MEB([(5,0,1), (-1,-3,4), (-1,-4,-3), (-1,4,-3)]))


[(0, 0, 0)]  ->  ((0, 0, 0), 0.0)
[(-1, 0, 0), (1, 0, 0)]  ->  ((0.0, 0.0, 0.0), 1.0)
[(-10, 0, 0), (10, 0, 0), (0, 1, 0)]  ->  ((0.0, -49.5, 0.0), 50.5)
[(-5, 0, 0), (3, -4, 0), (3, 4, 0)]  ->  ((0.0, 0.0, 0.0), 5.0)
[(5, 0, 1), (-1, -3, 4), (-1, -4, -3), (-1, 4, -3)]  ->  ((0.0, 0.0, 0.0), 5.0990195135927845)


### Question 2.

Pour construire le complexe tchèque, on énumère simplement les $(k+1)-$uplets et on calcule leur MEB, qui correspond au rayon minimal pour que des boules centrées en ces points aient un point en commun.

In [6]:
def max_cech(points,k):
    fichier = open("coord.txt", "w")   #pour pouvoir tracer le complexe
    for i in points:
        z=list(i)
        for r in range(len(z)):
            z[r]=str(z[r])
        fichier.write(' '.join(z)+'\n')
    fichier.close()
    x=[i for i in range(len(points))]
    simplexes=[{} for i in range(k+1)]  #liste de k dictionnaires, chacun contiendra les i-simplexes pour i<=k,
    fichier = open("cplx.txt", "w")     #ce qui facilitera l'optimisation à la prochaine question
    for i in range(k+1):
        for j in itertools.combinations(x,i+1):
            d=(MEB([points[k] for k in j]))[1]
            if d>=0:
                simplexes[i][j]=d
                z=list(j)
                for r in range(len(z)):
                    z[r]=str(z[r])
                fichier.write(' '.join(z)+'\n')
    fichier.close()
    return simplexes

def print_max_cech(points,k):
    simplexes=max_cech(points,k)
    for i in range(k+1):
        for k, v in sorted(simplexes[i].items(), key=lambda x: x[1]):
            print("%s -> [%s]" % (k, v))
        

Nous effectuons le test.

In [7]:
print_max_cech([(5,0,1), (-1,-3,4), (-1,-4,-3), (-1,4,-3)],3)

(0,) -> [0.0]
(1,) -> [0.0]
(2,) -> [0.0]
(3,) -> [0.0]
(1, 2) -> [3.535533905932738]
(0, 1) -> [3.6742346141747673]
(2, 3) -> [4.0]
(0, 2) -> [4.123105625617661]
(0, 3) -> [4.123105625617661]
(1, 3) -> [4.949747468305833]
(0, 1, 2) -> [4.395245364957663]
(0, 2, 3) -> [4.714951667914447]
(1, 2, 3) -> [5.0]
(0, 1, 3) -> [5.049752469181039]
(0, 1, 2, 3) -> [5.0990195135927845]


### Question 3.

L'algorithme naif serait:

In [8]:
def naive_cech(points,k, l):
    fichier = open("coord.txt", "w")
    for i in points:
        z=list(i)
        for r in range(len(z)):
            z[r]=str(z[r])
        fichier.write(' '.join(z)+'\n')
    fichier.close()
    x=[i for i in range(len(points))]
    simplexes=[{} for i in range(k+1)]
    d=0
    fichier = open("cplx.txt", "w")
    for i in range(k+1):
        for j in itertools.combinations(x,min(i+1,len(x))):
            d=(MEB([points[k] for k in j]))[1]
            if d<=l and d>=0:
                simplexes[i][j]=d
                z=list(j)
                for r in range(len(z)):
                    z[r]=str(z[r])
                fichier.write(' '.join(z)+'\n')
    fichier.close()
    return simplexes

Notre idée pour l'optimiser est de définir l'ensemble des $d-$voisins d'un complexe comme les points qui se trouvent à une distance au plus d des points de ce simplexes. Les distances sont directement données à un facteur 2 près par les MEB des simplexes de taille 2. On construit les $k-$simplexes à partir des $(k-1)-$simplexes en calculant les MEB des points d'un simplexe et d'un point qui lui est voisin, ce qui réduit considérablement les tests pour $l$ petit. Un discussion sur la notion de voisin est faite dans le rapport.

In [9]:
def cech(points, k, l):
    fichier = open("coord.txt", "w")
    for i in points:
        z=list(i)
        for r in range(len(z)):
            z[r]=str(z[r])
        fichier.write(' '.join(z)+'\n')
    fichier.close()
    fichier = open("cplx.txt", "w")
    x=[i for i in range(len(points))]
    d=0
    doublons=[]
    simplexes=[{} for i in range(k+1)]
    voisins=[{} for i in range(k+1)]
    for i in range(len(points)):
        simplexes[0][(i,)]=0
        voisins[0][(i,)]=set()                 #on utilise des sets pour éviter les répétitions 
        fichier.write(str(i)+'\n')             #et optimiser l'opération d'intersection
    for i,j in itertools.combinations(x,2):
        d= (MEB([points[i],points[j]]))[1]
        if d<=l and d>=0 :
            simplexes[1][(i,j)]=d
            z=[i,j]
            for r in range(len(z)):
                z[r]=str(z[r])
            fichier.write(' '.join(z)+'\n')
            voisins[0][(i,)].add(j)              #le calcul des MEB des simplexes de taille 2 nous permet 
            voisins[0][(j,)].add(i)              #de définir les voisins de chaque point
    for i in simplexes[1].keys():
        voisins[1][i]=(voisins[0][(i)[0],]).intersection(voisins[0][(i)[1],]) #on met facilement à jour la liste des voisins d'un 
    for i in range(2,k+1):                                                    #k-simplexe, qui n'est autre que l'intersection des
        doublons=[]                                                           #voisins du (k-1)-simplexe et du point dont il est issu
        for k,v in simplexes[i-1].items():
            for j in voisins[i-1][k]:           #on n'essaye d'ajouter que les points voisins du simplexe
                d=MEB([points[m] for m in (k+(j,))])[1]
                if set(k+(j,)) not in doublons and d<=l and d>=0:
                    doublons+=[set(k+(j,))]
                    simplexes[i][k+(j,)]=MEB([points[m] for m in (k+(j,))])[1]
                    voisins[i][k+(j,)]=(voisins[i-1][k]).intersection(voisins[0][(j,)]) 
                    z=list(k+(j,))
                    for r in range(len(z)):
                        z[r]=str(z[r])
                    fichier.write(' '.join(z)+'\n')
    fichier.close()
    return simplexes
                    
def print_cech(points,k, l):
    simplexe = cech(points,k, l)
    for i in range(k+1):
        for k, v in sorted(simplexe[i].items(), key=lambda x: x[1]):
            print("%s -> [%s]" % (k, v))

Effectuons un test.

In [10]:
print_cech([(5,0,1), (-1,-3,4), (-1,-4,-3), (-1,4,-3)],3, 4.5)

(0,) -> [0]
(1,) -> [0]
(2,) -> [0]
(3,) -> [0]
(1, 2) -> [3.535533905932738]
(0, 1) -> [3.6742346141747673]
(2, 3) -> [4.0]
(0, 2) -> [4.12310562561766]
(0, 3) -> [4.12310562561766]
(0, 1, 2) -> [4.395245364957663]


Pour effectuer les comparaisons de vitesse d'exécution, nous définissons la fonction suivante.

In [11]:
def cech_comp_time(points, k, l, N):
    dt1=0
    dt2=0
    for i in range(N):
        t1=time.time()
        naive_cech(points, k, l)
        dt1+=time.time()-t1
        t2=time.time()
        cech(points, k, l)
        dt2+=time.time()-t2
    return dt1/N,dt2/N

Tests: Nous allons génerer aléatoirement des points de l'espace et pour certaines valeurs de k et l, générer le simplexe de k points contraint à la filtration d'au plus l

In [12]:
N=20
k=3
tab=np.zeros((6*4,4))
S=np.array([-i for i in range(15)]+[i for i in range(15)])
L=np.array([i+0.5 for i in range(3,7)])
print("temps d'exécution t1 et t2 des deux algorithmes pour différentes valeurs de parametres n et l:")
i=0
for n in range(4,10):
    points=[tuple(np.random.choice(S, 3, replace = True))  for j in range(n)]
    #print(points)
    for l in L:
        #points=[np.random.choice(S, 3, replace = True).tolist()  for j in range(8)]
        print("Pour n= %s points et une filtration d'au plus l= %s" %(n,l))
        t1,t2=cech_comp_time(points, k, l, N)
        tab[i][0]=n
        tab[i][1]=l
        tab[i][2]=t1
        tab[i][3]=t2
        i=i+1
        print("t1=%s et t2=%s"%(t1,t2))
        print("--------------------------------------------------------------------------------------------------------")
#print(tab)

temps d'exécution t1 et t2 des deux algorithmes pour différentes valeurs de parametres n et l:
Pour n= 4 points et une filtration d'au plus l= 3.5
t1=0.004450571537017822 et t2=0.002099895477294922
--------------------------------------------------------------------------------------------------------
Pour n= 4 points et une filtration d'au plus l= 4.5
t1=0.004450571537017822 et t2=0.002099907398223877
--------------------------------------------------------------------------------------------------------
Pour n= 4 points et une filtration d'au plus l= 5.5
t1=0.00394970178604126 et t2=0.002200818061828613
--------------------------------------------------------------------------------------------------------
Pour n= 4 points et une filtration d'au plus l= 6.5
t1=0.004349470138549805 et t2=0.002000939846038818
--------------------------------------------------------------------------------------------------------
Pour n= 5 points et une filtration d'au plus l= 3.5
t1=0.00850048065185546

## $\alpha-$complexe

### Question 4

Nous utilisons l'algorithme de Seidel. Plus de détails dans le rapport

In [13]:
def a_filt(V,S):
    o,r=circum(S)
    i=0
    while r!=-1 and i<len(V):
        if dist(V[i],o)<r:
            o,r=a_filt(V[:i],S+[V[i]])
        i+=1
    return o,r

### Question 5

Nous utilisons à peu près le même algorithme que pour la question 3, avec l'$\alpha-$filtration à la place de la MEB.

In [14]:
def a_comp(points, k, l):
    fichier = open("coord.txt", "w")
    for i in points:
        z=list(i)
        for r in range(len(z)):
            z[r]=str(z[r])
        fichier.write(' '.join(z)+'\n')
    fichier.close()
    fichier = open("cplx.txt", "w")
    x=[i for i in range(len(points))]
    d=0
    doublons=[]
    simplexes=[{} for i in range(k+1)]
    voisins=[{} for i in range(k+1)]
    for i in range(len(points)):
        simplexes[0][(i,)]=0
        voisins[0][(i,)]=set()
        fichier.write(str(i)+'\n')
    for i,j in itertools.combinations(x,2):
        d= (a_filt(points,[points[i],points[j]]))[1]
        if d<=l and d>=0 :
            simplexes[1][(i,j)]=d
            z=[i,j]
            for r in range(len(z)):
                z[r]=str(z[r])
            fichier.write(' '.join(z)+'\n')
            voisins[0][(i,)].add(j)
            voisins[0][(j,)].add(i)
    for i in simplexes[1].keys():
        voisins[1][i]=(voisins[0][(i)[0],]).intersection(voisins[0][(i)[1],])
    for i in range(2,k+1):
        doublons=[]
        for k,v in simplexes[i-1].items():
            for j in voisins[i-1][k]:
                if set(k+(j,)) not in doublons:
                    d=a_filt(points,[points[m] for m in (k+(j,))])[1]
                    if d<=l and d>=0:
                        doublons+=[set(k+(j,))]
                        simplexes[i][k+(j,)]=d
                        voisins[i][k+(j,)]=(voisins[i-1][k]).intersection(voisins[0][(j,)])
                        z=list(k+(j,))
                        for r in range(len(z)):
                            z[r]=str(z[r])
                        fichier.write(' '.join(z)+'\n')
    fichier.close()
    return simplexes

def print_a_comp(points,k, l):
    simplexe = a_comp(points,k, l)
    for i in range(k+1):
        for k, v in sorted(simplexe[i].items(), key=lambda x: x[1]):
            print("%s -> [%s]" % (k, v))

Encore un test. Des figures sont disponibles dans le rapport.

In [15]:
print_a_comp([(5,0,1), (-1,-3,4), (-1,-4,-3), (-1,4,-3)],3, 4.5)

(0,) -> [0]
(1,) -> [0]
(2,) -> [0]
(3,) -> [0]
(0, 1) -> [3.6742346141747673]
(2, 3) -> [4.0]
(0, 2) -> [4.123105625617661]
(0, 3) -> [4.123105625617661]


## Complexe de Rips

### Question 6

Nous avons commencé par un algorithme assez naif: on trie les arêtes par ordre croissant de filtration, et à chaque étape on ajoute toutes les arêtes ayant la même filtration, puis on teste si l'une ou plusieurs d'entre elles sont dominées, auquel cas on les retire du graphe pour les réintroduire à l'étape suivante. La structure de données utilisée est une matrice d'adjacence, et la complexité est en $O(m.|E|.k^2)$ avec $k$ le degré maximal du graphe. Tout ceci est discuté dans le rapport.

In [16]:
def tab_aretes(G):              #renvoie la liste ordonnée des filtration et les arêtes correspondantes dans un dictionnaire
    t_rep=np.sort((np.array(G))[:,2])
    t=[t_rep[0]]
    for i in range(len(t_rep)-1):
        if t_rep[i]!=t_rep[i+1]:
            t+=[t_rep[i+1]]
    e={}
    for i in t:
        e[i]=[]
    for i in G:
        e[i[2]]+=[(i[0],i[1])]
    return t,e
        
def mat(G):                                                     #contruit une matrice d'adjacence vide 
    n=max([(max([G[i][0], G[i][1]])) for i in range(len(G))])   #(première étape, avant l'ajout de toute arête)
    M=[[0 for i in range(n+1)] for j in range(n+1)]
    return M,n

def ajouter(M,u,v):             #ajouter une arête à la matrice
    M[u][v]=1
    M[v][u]=1
    
def supprimer(M,u,v):           #supprimer une arête de la matrice
    M[u][v]=0
    M[v][u]=0
    
def commun(M,u,v,n):            #détermine les voisins communs aux deux sommets d'une arête
    commun=[u,v]
    for j in range(n+1):
        if M[u][j]!=0 and M[v][j]!=0:
            commun+=[j]
    return commun

def domination(M,u,v,n):        #détermine si une arête est dominée
    communs=commun(M,u,v,n)
    if len(communs)>2:          #s'il y a des voisins autres que u et v
        k=2
        fini=0
        while k<len(communs):
            l=0
            fini=0
            while l<len(communs) and fini==0:              #on ne teste que les sommets faisant partie des voisins, 
                if M[communs[k]][communs[l]]==0 and k!=l:  #car les autres n'ont pas à la fois u et v dans leur voisinage
                    fini=1
                l+=1
            if fini==0:
                return True
            k+=1
    return False

def simp(G):
    t,e=tab_aretes(G)
    m=len(t)
    M,n=mat(G)
    fichier = open("graphe.txt", "w")
    for i in range(m):
        for u,v in e[t[i]]:
            ajouter(M,u,v)             #il faut ajouter en même temps les arêtes ayant la même filtration. 
        for u,v in e[t[i]]:            #(Ce point est discuté dans le rapport)
            if domination(M,u,v,n):
                supprimer(M,u,v)
                if i!=m-1:
                    e[t[i+1]]+=[(u,v)]
            else:
                fichier.write(str(u)+" "+str(v)+" "+str(t[i])+"\n")
    fichier.close()
    return M

Pour tester notre fonction, on écrit les codes simples suivants.

In [17]:
def complet(n):
    d=1
    g=[]
    x=[]
    x=[i for i in range(n)]
    fichier = open("complet.txt", "w")
    for i in itertools.combinations(x,2):
        z=list(i)
        x=z.copy()
        for r in range(2):
            z[r]=str(z[r])
        x+=[d]
        fichier.write(' '.join(z)+" "+'1'+'\n')
        g+=[x]
    fichier.close()
    return(g)

def polygon(n):
    g=[]
    x=[]
    points=[[m.sin(i*2*m.pi/(2*n)),m.cos(i*2*m.pi/(2*n))] for i in range((2*n))]
    x=[i for i in range((2*n))]
    fichier = open("polygon.txt", "w")
    for i in itertools.combinations(x,2):
        z=list(i)
        x=z.copy()
        d=round(dist(points[z[0]],points[z[1]]),5) #pour ne pas différiencer, par exemple, les valeurs 0.9999999... et 1.0
        for r in range(2):
            z[r]=str(z[r])
        x+=[d]
        fichier.write(' '.join(z)+" "+str(d)+'\n')
        g+=[x]
    fichier.close()
    return(g)

def alea(n):
    g=[]
    x=[]
    points=[[random.random(),random.random()] for i in range(n)]
    x=[i for i in range(n)]
    fichier = open("alea.txt", "w")
    for i in itertools.combinations(x,2):
        z=list(i)
        x=z.copy()
        d=round(dist(points[z[0]],points[z[1]]),5)
        for r in range(2):
            z[r]=str(z[r])
        x+=[d]
        fichier.write(' '.join(z)+" "+str(d)+'\n')
        g+=[x]
    fichier.close()
    return(g)

On peut faire quelques tests.

In [18]:
simp(complet(10))

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 0]]

In [19]:
simp(polygon(5))

[[0, 1, 1, 1, 1, 0, 1, 1, 1, 1],
 [1, 0, 1, 1, 1, 1, 0, 1, 1, 1],
 [1, 1, 0, 1, 1, 1, 1, 0, 1, 1],
 [1, 1, 1, 0, 1, 1, 1, 1, 0, 1],
 [1, 1, 1, 1, 0, 1, 1, 1, 1, 1],
 [0, 1, 1, 1, 1, 0, 1, 1, 1, 1],
 [1, 0, 1, 1, 1, 1, 0, 1, 1, 1],
 [1, 1, 0, 1, 1, 1, 1, 0, 1, 1],
 [1, 1, 1, 0, 1, 1, 1, 1, 0, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 0]]

In [20]:
simp(alea(10))

[[0, 0, 0, 1, 1, 1, 1, 1, 1, 0],
 [0, 0, 0, 0, 1, 0, 1, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 1, 0, 0],
 [1, 1, 0, 0, 0, 0, 1, 0, 1, 0],
 [1, 0, 0, 0, 0, 0, 1, 1, 0, 1],
 [1, 1, 0, 0, 1, 1, 0, 0, 0, 0],
 [1, 0, 1, 1, 0, 1, 0, 0, 0, 1],
 [1, 0, 0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 1, 0, 0]]

Notre algoritme donne la solution optimale mais est trop lent, notamment pour un nombre de filtrations différentes élevé. En effet, chaque arête peut être réintroduite $m$ fois, et il faut refaire le teste de domination à chaque fois. Notre idée pour l'optimiser est d'utiliser l'observation simple suivante: si à une étape $i+1$ toutes les arêtes de filtration $initiale$ $i+1$ sont dominées, alors toutes celles héritées des étapes précédentes sont elles aussi dominées. Encore une fois, il faut se référer au rapport pour plus de détails.

In [21]:
def tab_aretes_opt(G):
    t_rep=np.sort((np.array(G))[:,2])
    t=[t_rep[0]]
    for i in range(len(t_rep)-1):
        if t_rep[i]!=t_rep[i+1]:
            t+=[t_rep[i+1]]
    e={}
    for i in t:
        e[i]=[[],[]]   #ou alloue une place aux arêtes qui seront héritées des étapes précédentes
    for i in G:
        e[i[2]][0]+=[(i[0],i[1])]
    return t,e

def simp_opt(G):
    t,e=tab_aretes_opt(G)
    m=len(t)
    M,n=mat(G)
    dom=0
    fichier = open("graphe.txt", "w")
    for i in range(m):
        for u,v in e[t[i]][1]:
            ajouter(M,u,v)
        for u,v in e[t[i]][0]:
            ajouter(M,u,v)  #il faut tout ajouter avant!
        dom=1
        for u,v in e[t[i]][0]:
            if domination(M,u,v,n):
                supprimer(M,u,v)
                if i!=m-1:
                    e[t[i+1]][1]+=[(u,v)]
            else:
                dom=0
                fichier.write(str(u)+" "+str(v)+" "+str(t[i])+"\n")
        if dom==0:
            for u,v in e[t[i]][1]:
                if domination(M,u,v,n):
                    supprimer(M,u,v)
                    if i!=m-1:
                        e[t[i+1]][1]+=[(u,v)]
                else:
                    fichier.write(str(u)+" "+str(v)+" "+str(t[i])+"\n")
        else:
            for u,v in e[t[i]][1]:
                    supprimer(M,u,v)
            if i!=m-1:
                e[t[i+1]][1]+=e[t[i]][1]
    return M

Les temps d'éxecution sont nettement meilleurs. Nous utilisons le code suivant pour les comparer.

In [22]:
def simp_time(G,N):
    dt1=0
    dt2=0
    for i in range(N):
        t1=time.time()
        simp(G)
        dt1+=time.time()-t1
        t2=time.time()
        simp_opt(G)
        dt2+=time.time()-t2
    return dt1/N,dt2/N  

Nous commentons nos résultats dans le rapport.

In [23]:
tab2=np.zeros((8,3))
tab3=np.zeros((8,3))
print("Complete graph")
N=10
i=0
for n in range (40,100,10):
    print("n=%s"%n)
    t=simp_time(complet(n),N)
    print(t)
    tab2[i][0]=t[1]
    tab3[i][0]=t[0]
    i=i+1

Complete graph
n=40
(0.011400628089904784, 0.011300969123840331)
n=50
(0.019301295280456543, 0.0181016206741333)
n=60
(0.031800460815429685, 0.03100435733795166)
n=70
(0.048192811012268064, 0.04560556411743164)
n=80
(0.06460511684417725, 0.06860625743865967)
n=90
(0.09380991458892822, 0.09210696220397949)


In [24]:
print("Regular 2n-gon")
N=10
i=0
for n in range (40,100,10):
    print("n=%s"%n)
    t=simp_time(polygon(n//2),N)
    print(t)
    tab2[i][1]=t[1]
    tab3[i][1]=t[0]
    i=i+1

Regular 2n-gon
n=40
(0.09290421009063721, 0.07396705150604248)
n=50
(0.18589205741882325, 0.14401166439056395)
n=60
(0.3689899206161499, 0.278573203086853)
n=70
(0.6545783758163453, 0.4847155809402466)
n=80
(1.2544896841049193, 0.9227670669555664)
n=90
(1.7064910888671876, 1.2608901262283325)


In [26]:
print("Random points in a square")
N=5
i=0
for n in range (40,100,10):
    print("n=%s"%n)
    t=simp_time(alea(n),N)
    print(t)
    tab2[i][2]=t[1]
    tab3[i][2]=t[0]
    i=i+1

Random points in a square
n=40
(2.9291535377502442, 0.15020923614501952)
n=50
(8.5924391746521, 0.346342658996582)
n=60
(22.06142406463623, 0.7529832363128662)
n=70
(43.36151781082153, 1.3319105148315429)
n=80
(82.78685607910157, 2.2603389263153075)
n=90
(145.35683336257935, 3.6109035491943358)
